In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from statsmodels.stats.stattools import durbin_watson
import statsmodels.api as sm

import matplotlib
matplotlib.use('TkAgg')  # или 'Qt5Agg'

# Данные
data = {
    'День': list(range(1, 13)),
    'Доллар': [28.12, 28.18, 28.13, 28.08, 28.06, 28.03, 28.02, 28.00, 27.99, 27.93, 27.95, 27.97],
    'Евро': [36.13, 35.97, 35.97, 36.00, 36.13, 36.28, 36.34, 36.47, 36.54, 36.50, 36.52, 36.54],
    'Иена': [26.97, 26.80, 26.77, 26.63, 26.53, 26.70, 26.67, 26.63, 26.60, 26.50, 26.55, 26.52],
    'Фунт': [52.63, 52.32, 52.26, 52.28, 52.43, 52.58, 52.90, 52.99, 52.81, 52.89, 52.62, 52.67]
}

df = pd.DataFrame(data)

# Функция для теста Фаррара – Глоубера
def farra_glob_test(X, y):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()
    return model

# Проверка мультиколлинеарности между евро, иеной и фунтом
variables = ['Евро', 'Иена', 'Фунт']
results = {}

for var in variables:
    X = df[variables].drop(var, axis=1)  # Убираем одну переменную
    y = df[var]
    model = farra_glob_test(X, y)
    results[var] = model.summary()

# Вывод результатов
for var, result in results.items():
    print(f"Результаты для {var}:\n{result}\n")

# Линейная регрессия
X = df[['Евро', 'Иена', 'Фунт']]
y = df['Доллар']
X = sm.add_constant(X)  # добавляем константу

model = sm.OLS(y, X).fit()
print(model.summary())

# Влияние факторов
coefficients = model.params[1:]  # исключаем константу
print("Коэффициенты влияния факторов:")
print(coefficients)

Результаты для Евро:
                            OLS Regression Results                            
Dep. Variable:                   Евро   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.791
Method:                 Least Squares   F-statistic:                     21.84
Date:                Sun, 23 Feb 2025   Prob (F-statistic):           0.000352
Time:                        09:38:00   Log-Likelihood:                 11.584
No. Observations:                  12   AIC:                            -17.17
Df Residuals:                       9   BIC:                            -15.71
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.4818     10.92

C:\Users\nikit\PycharmProjects\Econometric1\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=12 observations were given.
  return hypotest_fun_in(*args, **kwds)
C:\Users\nikit\PycharmProjects\Econometric1\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=12 observations were given.
  return hypotest_fun_in(*args, **kwds)
C:\Users\nikit\PycharmProjects\Econometric1\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=12 observations were given.
  return hypotest_fun_in(*args, **kwds)
C:\Users\nikit\PycharmProjects\Econometric1\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=

In [4]:
# Остатки модели
residuals = model.resid
dw_stat = durbin_watson(residuals)
print(f"Статистика Дурбина-Уотсона: {dw_stat}")

# Функция для построения и вывода результатов регрессионной модели
def run_regression(dependent_var, independent_vars):
    X = df[independent_vars]
    y = df[dependent_var]
    X = sm.add_constant(X)  # добавляем константу
    model = sm.OLS(y, X).fit()
    print(f"Результаты регрессии для {dependent_var} от {', '.join(independent_vars)}:\n")
    print(model.summary())
    print("\n" + "="*80 + "\n")
    # Графики
    # остатков
    residuals = model.resid

    # Гистограмма остатков
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.hist(residuals, bins=10, edgecolor='k', alpha=0.7)
    plt.title('Гистограмма остатков')
    plt.xlabel('Остатки')
    plt.ylabel('Частота')

    # Q-Q график
    plt.subplot(1, 2, 2)
    stats.probplot(residuals, dist="norm", plot=plt)
    plt.title('Q-Q график остатков')

    plt.tight_layout()
    plt.show()

    return model

# 1. Регрессия: Доллар от Евро
model_euro = run_regression('Доллар', ['Евро'])

# 2. Регрессия: Доллар от Иены
model_yen = run_regression('Доллар', ['Иена'])

# 3. Регрессия: Доллар от Иены и Фунта
model_yen_pound = run_regression('Доллар', ['Иена', 'Фунт'])

Статистика Дурбина-Уотсона: 2.4221196415107205
Результаты регрессии для Доллар от Евро:

                            OLS Regression Results                            
Dep. Variable:                 Доллар   R-squared:                       0.856
Model:                            OLS   Adj. R-squared:                  0.842
Method:                 Least Squares   F-statistic:                     59.49
Date:                Sun, 23 Feb 2025   Prob (F-statistic):           1.62e-05
Time:                        09:38:14   Log-Likelihood:                 25.844
No. Observations:                  12   AIC:                            -47.69
Df Residuals:                      10   BIC:                            -46.72
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------

C:\Users\nikit\PycharmProjects\Econometric1\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=12 observations were given.
  return hypotest_fun_in(*args, **kwds)


Результаты регрессии для Доллар от Иена:

                            OLS Regression Results                            
Dep. Variable:                 Доллар   R-squared:                       0.607
Model:                            OLS   Adj. R-squared:                  0.568
Method:                 Least Squares   F-statistic:                     15.44
Date:                Sun, 23 Feb 2025   Prob (F-statistic):            0.00282
Time:                        09:38:21   Log-Likelihood:                 19.815
No. Observations:                  12   AIC:                            -35.63
Df Residuals:                      10   BIC:                            -34.66
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const     

C:\Users\nikit\PycharmProjects\Econometric1\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=12 observations were given.
  return hypotest_fun_in(*args, **kwds)


Результаты регрессии для Доллар от Иена, Фунт:

                            OLS Regression Results                            
Dep. Variable:                 Доллар   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     28.24
Date:                Sun, 23 Feb 2025   Prob (F-statistic):           0.000132
Time:                        09:38:24   Log-Likelihood:                 26.119
No. Observations:                  12   AIC:                            -46.24
Df Residuals:                       9   BIC:                            -44.78
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cons

C:\Users\nikit\PycharmProjects\Econometric1\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=12 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [5]:
# Прогнозирование курса доллара на следующие два дня
# Предположим, что курсы евро, иены и фунта на следующие два дня будут следующими:
future_data = pd.DataFrame({
    'Евро': [36.55, 36.60],  # Прогнозируемые значения
    'Иена': [26.55, 26.50],
    'Фунт': [52.70, 52.75]
})

# Прогноз для модели Доллар от Евро
future_data_with_const = sm.add_constant(future_data[['Евро']])
predictions_euro = model_euro.predict(future_data_with_const)

# Прогноз для модели Доллар от Иены
predictions_yen = model_yen.predict(sm.add_constant(future_data[['Иена']]))

# Прогноз для модели Доллар от Иены и Фунта
predictions_yen_pound = model_yen_pound.predict(sm.add_constant(future_data[['Иена', 'Фунт']]))

# Вывод прогнозов
print("Прогноз курса доллара на следующие два дня (от Евро):", predictions_euro)
print("Прогноз курса доллара на следующие два дня (от Иены):", predictions_yen)
print("Прогноз курса доллара на следующие два дня (от Иены и Фунта):", predictions_yen_pound)


Прогноз курса доллара на следующие два дня (от Евро): 0    27.956145
1    27.940782
dtype: float64
Прогноз курса доллара на следующие два дня (от Иены): 0    27.992068
1    27.970211
dtype: float64
Прогноз курса доллара на следующие два дня (от Иены и Фунта): 0    27.987750
1    27.962252
dtype: float64
